In [1]:
from sample_bathy import SampleBathy
from ipywidgets import IntText, FloatText, Output, Button, IntSlider, Dropdown
from ipywidgets import Box, VBox, HBox, Label, Layout, Text, BoundedIntText
from IPython.display import HTML
import glob

In [2]:
be = SampleBathy()
be.read_survey_array_from_file('surveys.nc')

n_ens = IntText(value=100)
n_vec = IntText(value=10)
stdev = FloatText(value=1.0)
output1 = Output()
output2 = Output()
output3 = Output()
output4 = Output()

ensemble_member = BoundedIntText(
    description='Member:', min=0, max=n_ens.value-1, layout=Layout(width='148px'))
survey_member = BoundedIntText(
    description='Member:', min=0, max=be.survey_array.shape[1]-1, layout=Layout(width='148px'))

def generate_ensemble(click):
    be.generate_ensemble(n_ens.value, stdev=stdev.value, n_svec=n_vec.value)
        
run_button = Button(description='Generate Ensemble')
run_button.on_click(generate_ensemble)

def plot_survey_mean(click):
    output1.clear_output()
    be.compute_survey_mean()
    with output1:
        be.plot(be.survey_mean, title='Survey mean')
    
survey_mean_button = Button(description='Plot survey mean')
survey_mean_button.on_click(plot_survey_mean)

def plot_ensemble_mean(click):
    output2.clear_output()
    with output2:
        try:
            be.compute_ensemble_mean()
        except:
            print('Generate ensemble above.')
            return
        be.plot(be.ensemble_mean, title='Ensemble mean')

ensemble_mean_button = Button(description='Plot ensemble mean')
ensemble_mean_button.on_click(plot_ensemble_mean)

def plot_ensemble_member(click):
    output3.clear_output()
    member = ensemble_member.value
    with output3:
        try:
            be.plot(be.ensemble_array[:,member], title='Ensemble {}'.format(member))
        except:
            print('Generate ensemble above.')

ensemble_member_button = Button(description='Plot ens. member')
ensemble_member_button.on_click(plot_ensemble_member)

def plot_survey_member(click):
    output4.clear_output()
    member = survey_member.value
    with output4:
        be.plot(be.survey_array[:,member], title='Survey {}'.format(member))

survey_member_button = Button(description='Plot survey member')
survey_member_button.on_click(plot_survey_member)

def save_ensemble(click):
    filename = output_filename.value
    output5.clear_output()
    with output5:
        try:
            be.ensemble_array
        except:
            print('Generate ensemble above')
            return
        be.write_ensemble_array_to_nc(filename)
        
file_input = Dropdown(options=glob.glob('*.nc'))

def load_ensemble(click):
    filename = file_input.value
    be.read_ensemble_array_from_file(filename)
    
load_button = Button(description='Load ensemble')
load_button.on_click(load_ensemble)

save_button = Button(description='Save ensemble')
save_button.on_click(save_ensemble)

output_filename = Text(description='Filename:', value='ensemble.nc')

output5 = Output()

### Overview
This notebook generates and ensemble of bathymetry realizations informed by the FRF integrated bathymetry product (Young, Bak, and Forte, 2018) from between 2006 and 2019. An anomaly array is constructed from the integrated bathymetry array, and a singular value decomposition (SVD) is performed on the anomaly array. New bathymetry realizations are produced by taking combinations of the principle modes of the SVD. The user may select the number of principle modes and the standard deviation of the standard normal distribution used to determine the weights assigned to each mode.

### Generate Ensemble or Load Existing Ensemble

In [4]:
HBox(
    [VBox(
        [Label('Ensemble members'),
         n_ens,
         Label('Number of modes for sampling'),
         n_vec,
         Label('Standard deviation of distribution'),
         stdev,
         run_button]), 
     Box([], layout=Layout(width='50px')),
     VBox(
         [Box([], layout=Layout(height='128px')),
          Label('Select ensemble file'),
          file_input, 
          load_button])])

### Visualization

In [5]:
HBox(
    [VBox([survey_mean_button, Box([], layout=Layout(height='33px')), output1]), 
     VBox([ensemble_mean_button, Box([], layout=Layout(height='33px')), output2]), 
     VBox([ensemble_member_button, ensemble_member, output3]),
     VBox([survey_member_button, survey_member, output4])])

### Save Ensemble
The ensemble array is saved to a netCDF file in the same directory as the notebook. Existing files are not overwritten.

In [7]:
VBox(
    [HBox([save_button, output_filename]),
     output5])

In [8]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')